In [1]:
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
ret_day = pyreadr.read_r(r'C:\Users\Lenovo\Desktop\data\Output\ret_day2023.RDS')
ret_day = ret_day[None]
ret_day['Day'] = pd.to_datetime(ret_day['Day'],format='%Y-%m-%d') # change the format of time
# ret_day.set_index(['Day'],inplace=True) # set the index
ret_day 
#123

,Day,Stkcd,y,Q,month,Opnprc,Hiprc,Loprc,Clsprc,TradingVolume,...,H_float_shares,Pre_shares,ratio,Freq,Freq_Q,Freq_y,yuemo,jimo,rfday,r
0,1990-12-19,600601,1990.0,1990.75,1990.916667,185.3,185.3,185.3,185.3,200.0,...,0.0,0.0,0.910000,8,8,8,0.0,0.0,0.000227,2.705773
1,1990-12-19,600602,1990.0,1990.75,1990.916667,365.7,384.0,365.7,384.0,1160.0,...,0.0,0.0,0.245500,8,8,8,0.0,0.0,0.000227,2.839773
2,1990-12-19,600651,1990.0,1990.75,1990.916667,320.3,320.3,320.3,320.3,2.0,...,0.0,0.0,1.000000,8,8,8,0.0,0.0,0.000227,3.575487
3,1990-12-19,600656,1990.0,1990.75,1990.916667,260.0,260.0,260.0,260.0,50.0,...,0.0,0.0,0.182488,5,5,5,0.0,0.0,0.000227,1.599773
4,1990-12-20,600601,1990.0,1990.75,1990.916667,185.3,194.6,185.3,194.6,84.0,...,0.0,0.0,0.910000,8,8,8,0.0,0.0,0.000227,0.049962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13597254,2023-12-29,605580,2023.0,2023.75,2023.916667,11.45,11.64,11.39,11.62,1443740.0,...,0.0,0.0,0.255550,21,60,242,1.0,1.0,0.000041,0.016582
13597255,2023-12-29,605588,2023.0,2023.75,2023.916667,49.86,52.48,49.76,51.7,3720800.0,...,0.0,0.0,0.313813,21,60,242,1.0,1.0,0.000041,0.035201
13597256,2023-12-29,605589,2023.0,2023.75,2023.916667,22.19,22.59,22.12,22.38,5546616.0,...,0.0,0.0,0.784412,21,60,242,1.0,1.0,0.000041,0.008521
13597257,2023-12-29,605598,2023.0,2023.75,2023.916667,25.59,26.76,25.59,26.76,1463545.0,...,0.0,0.0,0.299888,21,60,242,1.0,1.0,0.000041,0.037973


In [ ]:
#从ret_day随机选择300只股票计算其circulationvalue加权的Return_1
# 重复1000次

# ret[Stkcd] 提取出来，去掉重复值
stkcd = ret_day['Stkcd'].drop_duplicates()

# 从stkcd中随机选取300只股票
stkcd_300 = stkcd.sample(n=300)

# 从ret_day中提取出stkcd_300的数据
ret_day_300 = ret_day[ret_day['Stkcd'].isin(stkcd_300)]

# 每日计算Circulationvalue加权的Return_1
# 使用groupby
# ret_day_300['CirculationValue'] = ret_day_300['CirculationValue'].astype(float)
ret_sample = ret_day_300.groupby('Day').apply(lambda x: np.sum(x['CirculationValue'] * x['Return_1']) / np.sum(x['CirculationValue']))

# 转换成dataframe
# ret_sample = ret_sample.to_frame()
ret_sample = pd.DataFrame(ret_sample)

# 重命名列名
ret_sample.columns = ['return']

# 计算季度波动率 resample
ret_sample_quarter = ret_sample.resample('Q').apply(lambda x: np.sum(x['return']**2))

# rename index to Q
ret_sample_quarter.index = ret_sample_quarter.index.rename('Q')

ret_sample_quarter

In [ ]:
# 重复上述步骤1000次

# ret[Stkcd] 提取出来，去掉重复值
stkcd = ret_day['Stkcd'].drop_duplicates()

# 用于存储结果的列表
all_ret_samples = []

# 重复1000次
for _ in range(100):
    # 从stkcd中随机选取300只股票
    stkcd_300 = stkcd.sample(n=500, replace=False)

    # 从ret_day中提取出stkcd_300的数据
    ret_day_300 = ret_day[ret_day['Stkcd'].isin(stkcd_300)]

    # 每日计算Circulationvalue加权的Return_1
    ret_sample = ret_day_300.groupby('Day').apply(
        lambda x: np.sum(x['CirculationValue'].astype(float) * x['Return_1']) / np.sum(x['CirculationValue'].astype(float))
    )

    # 转换成dataframe并重命名列名
    ret_sample_df = pd.DataFrame(ret_sample, columns=['return'])

    # 计算季度波动率 resample
    ret_sample_quarter = ret_sample_df['return'].resample('Q').apply(lambda x: np.sum(x**2))

    # rename index to Q
    ret_sample_quarter.index = ret_sample_quarter.index.rename('Q')
    
    # 将单次结果添加到列表中
    all_ret_samples.append(ret_sample_quarter)

# 合并所有季度波动率数据到一个DataFrame
combined_quarter_volatility = pd.concat(all_ret_samples, axis=1)

# 保留2000年后的样本
combined_quarter_volatility = combined_quarter_volatility['2000-01-01':]
combined_quarter_volatility

In [ ]:
# combined_quarter_volatility的每一列画在同一张图上
plt.figure(figsize=(12, 6))

# Now plot each stock's feature value over time
for column in combined_quarter_volatility.columns:
    plt.plot(combined_quarter_volatility.index, combined_quarter_volatility[column], label=f'Sample {column}')
    
# Get current axis of the plot.
ax = plt.gca()

# 设置x轴的日期格式
date_format = mdates.DateFormatter('%Y')
ax.xaxis.set_major_formatter(date_format)

# 设置x轴的刻度间隔
ax.xaxis.set_major_locator(mdates.YearLocator())

# 旋转x轴标签以防止重叠
plt.xticks(rotation=90)
plt.savefig('Pic/random_vol.pdf', bbox_inches='tight')# 更改输出图片格式 jpg
plt.show();